<a href="https://colab.research.google.com/github/prawizard/TweetsClassification_NLP/blob/main/TweetEval/TweetEval_IronyTweetsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install/import the required libraries

In [24]:
%%time
!pip install ktrain

CPU times: user 23.1 ms, sys: 43.3 ms, total: 66.5 ms
Wall time: 2.6 s


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
import re
import requests
import sklearn.metrics

# Define Constants

In [3]:
TRAIN_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/train_text.txt"
TRAIN_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/train_labels.txt"
VAL_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/val_text.txt"
VAL_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/val_labels.txt"
TEST_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/test_text.txt"
TEST_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/test_labels.txt"
VOCAB_SIZE=2000

# Fetch the Data using URLs

In [4]:
r = requests.get(TRAIN_TEXT_URL, allow_redirects=True)
open('train_text.txt', 'wb').write(r.content)

r = requests.get(TRAIN_LABELS_URL, allow_redirects=True)
open('train_labels.txt', 'wb').write(r.content)

r = requests.get(VAL_TEXT_URL, allow_redirects=True)
open('val_text.txt', 'wb').write(r.content)

r = requests.get(VAL_LABELS_URL, allow_redirects=True)
open('val_labels.txt', 'wb').write(r.content)

r = requests.get(TEST_TEXT_URL, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r = requests.get(TEST_LABELS_URL, allow_redirects=True)
open('test_labels.txt', 'wb').write(r.content)

1568

# Access the data from files

In [5]:
stream=open("train_text.txt")
tweets=stream.readlines()
stream.close()
val_stream=open("val_text.txt")
val_tweets=val_stream.readlines()
val_stream.close()
test_stream=open("test_text.txt")
test_tweets=test_stream.readlines()
test_stream.close()

# Labels
stream=open("train_labels.txt")
tweetsLabels=stream.readlines()
stream.close()
val_stream=open("val_labels.txt")
val_tweetsLabels=val_stream.readlines()
val_stream.close()
test_stream=open("test_labels.txt")
test_tweetsLabels=test_stream.readlines()
test_stream.close()


# Labels
labels=[0]*len(tweetsLabels)
for i in range(len(tweetsLabels)):
  if tweetsLabels[i].find('\n')!=-1:
    labels[i]=int(re.sub('\n', '', tweetsLabels[i]))
val_labels=[0]*len(val_tweetsLabels)
for i in range(len(val_tweetsLabels)):
  if val_tweetsLabels[i].find('\n')!=-1:
    val_labels[i]=int(re.sub('\n', '', val_tweetsLabels[i]))
test_labels=[0]*len(test_tweetsLabels)
for i in range(len(test_tweetsLabels)):
  if test_tweetsLabels[i].find('\n')!=-1:
    test_labels[i]=int(re.sub('\n', '', test_tweetsLabels[i]))

In [6]:
print('Samples in Training set : ',len(labels),', Validation set : ', len(val_labels),', Test set : ', len(test_labels))

Samples in Training set :  2862 , Validation set :  955 , Test set :  784


# Data Cleaning

## Remove the twitter handles

In [7]:
for i in range(len(tweets)):
  if tweets[i].find('@user')!=-1:
    tweets[i]=re.sub('@user', '', tweets[i])
    tweets[i]=re.sub('#+', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('@user')!=-1:
    val_tweets[i]=re.sub('@user', '', val_tweets[i])
    val_tweets[i]=re.sub('#+', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('@user')!=-1:
    test_tweets[i]=re.sub('@user', '', test_tweets[i])
    test_tweets[i]=re.sub('#+', '', test_tweets[i])

## Remove the unnecessary hashtags

In [8]:
for i in range(len(tweets)):
  if tweets[i].find('#[a-zA-Z]+')!=-1:
    tweets[i]=re.sub('#[a-zA-Z]+', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('#[a-zA-Z]+')!=-1:
    val_tweets[i]=re.sub('#[a-zA-Z]+', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('#[a-zA-Z]+')!=-1:
    test_tweets[i]=re.sub('#[a-zA-Z]+', '', test_tweets[i])

## Remove characters like \\n and unnecessary dots



In [9]:
for i in range(len(tweets)):
  if tweets[i].find('\n')!=-1:
    tweets[i]=re.sub('\n', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('\n')!=-1:
    val_tweets[i]=re.sub('\n', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('\n')!=-1:
    test_tweets[i]=re.sub('\n', '', test_tweets[i])

# Unnecessary dots
p='\.\.\.|\.\.'

for i in range(len(tweets)):
  tweets[i]=re.sub(p, '', tweets[i])
  tweets[i]=tweets[i].lower()

for i in range(len(val_tweets)):
  val_tweets[i]=re.sub(p, '', val_tweets[i])
  val_tweets[i]=val_tweets[i].lower()

for i in range(len(test_tweets)):
  test_tweets[i]=re.sub(p, '', test_tweets[i])
  test_tweets[i]=test_tweets[i].lower()

# Store the Data Processed In a Data Frame

In [10]:
rows=[]
rowIndices=[]
for i in range(len(tweets)):
  rows.append({"TWEET":tweets[i], "CATEGORY":labels[i]})
  rowIndices.append(i+1)
train_df=pd.DataFrame(rows, index=rowIndices)

val_rows=[]
val_rowIndices=[]
for i in range(len(val_tweets)):
  val_rows.append({"TWEET":val_tweets[i], "CATEGORY":val_labels[i]})
  val_rowIndices.append(i+1)
val_df=pd.DataFrame(val_rows, index=val_rowIndices)

test_rows=[]
test_rowIndices=[]
for i in range(len(test_tweets)):
  test_rows.append({"TWEET":test_tweets[i], "CATEGORY":test_labels[i]})
  test_rowIndices.append(i+1)
test_df=pd.DataFrame(test_rows, index=test_rowIndices)

In [11]:
train_df=pd.concat([train_df, train_df])
train_df

,TWEET,CATEGORY
1,seeing ppl walking w/ crutches makes me really...,1
2,"look for the girl with the broken smile, ask h...",0
3,now i remember why i buy books online service...,1
4,so is he banded from wearing the clothes? k...,1
5,just found out there are etch a sketch apps. ...,1
...,...,...
2858,i don't have to respect your beliefs.||i only ...,0
2859,women getting hit on by married managers at c...,1
2860,no but i followed you and i saw you posted th...,0
2861,i dont know what it is but i'm in love your p...,0


# Segregate the Data Into Training and Validation/Dev Set

In [12]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=train_df,
                                                                   text_column = 'TWEET',
                                                                   label_columns = 'CATEGORY',
                                                                   val_df = val_df,
                                                                   maxlen = 65,
                                                                   preprocess_mode = 'bert')

['not_CATEGORY', 'CATEGORY']
   not_CATEGORY  CATEGORY
1           0.0       1.0
2           1.0       0.0
3           0.0       1.0
4           0.0       1.0
5           0.0       1.0
['not_CATEGORY', 'CATEGORY']
   not_CATEGORY  CATEGORY
1           0.0       1.0
2           1.0       0.0
3           0.0       1.0
4           0.0       1.0
5           0.0       1.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


# * For loaded weights, skip all the cells below until the cell " (Running the Model Using Weights (Pretrained)) and run all the cells that follow thereafter " *

# Using BERT for pretrained weights

In [13]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 65
done.


# Tuning the hyper parameters 

In [14]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 4)

# Fit the Model

In [15]:
learner.fit_onecycle(lr = 2e-5, epochs = 2)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/My Drive/bert')



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
1431/1431 [==============================] - 419s 282ms/step - loss: 0.6477 - accuracy: 0.5997 - val_loss: 0.6136 - val_accuracy: 0.6576
Epoch 2/2
1431/1431 [==============================] - 426s 298ms/step - loss: 0.2454 - accuracy: 0.9037 - val_loss: 0.8633 - val_accuracy: 0.6953


# Evaluate the Model Performance on Test Data

In [16]:
data = test_df['TWEET'].tolist()

In [17]:
bert_pred=predictor.predict(data)

In [19]:
y_true=np.array(test_df['CATEGORY'].tolist())

In [20]:
y_true[:15]

array([0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0])

In [21]:
res_bert=[]
for i in range(len(bert_pred)):
  if bert_pred[i]=='CATEGORY':
    res_bert.append(1)
  else:
    res_bert.append(0)

# Accuracy and F1-Score

In [25]:
res_bert=np.array(res_bert)
print('ACCURACY OBTAINED :', round(np.mean(res_bert==y_true)*100,2))

ACCURACY OBTAINED : 71.43


In [23]:
F1_SCORE=sklearn.metrics.f1_score(y_true, res_bert, average='macro')
print('F1_SCORE OBTAINED :', round(F1_SCORE*100, 2))

F1_SCORE OBTAINED : 71.1


# Saving the Trained Weights

In [27]:
predictor.save('/content/TrainedWeights/Weights')

In [39]:
# !zip -r /content/TrainedWeights/Weights.zip /content/TrainedWeights/Weights
!zip -r /content/Irony_Weights.zip /content/TrainedWeights/Weights

  adding: content/TrainedWeights/Weights/ (stored 0%)
  adding: content/TrainedWeights/Weights/tf_model.h5 (deflated 18%)
  adding: content/TrainedWeights/Weights/tf_model.preproc (deflated 52%)


In [43]:
from google.colab import files
files.download('/content/Irony_Weights.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Running the Model Using Weights (Pretrained)

Upload the zip file 'Irony_Weights.zip' in the runtime and Execute the Below Cells

In [31]:
predictor_load = ktrain.load_predictor('/content/Irony_Weights')

## Evaluate on Test Data Using these loaded weights

In [32]:
data = test_df['TWEET'].tolist()

In [33]:
bert_pred_load=predictor_load.predict(data)

In [34]:
y_true=np.array(test_df['CATEGORY'].tolist())

In [35]:
res_bert_load=[]
for i in range(len(bert_pred_load)):
  if bert_pred_load[i]=='CATEGORY':
    res_bert_load.append(1)
  else:
    res_bert_load.append(0)

## ACCURACY and F1_SCORE

In [36]:
res_bert_load=np.array(res_bert_load)
print('ACCURACY OBTAINED :', round(np.mean(res_bert_load==y_true)*100,2))

ACCURACY OBTAINED : 71.43


In [37]:
F1_SCORE=sklearn.metrics.f1_score(y_true, res_bert_load, average='macro')
print('F1_SCORE OBTAINED :', round(F1_SCORE*100, 2))

F1_SCORE OBTAINED : 71.1
